In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
from eegatscale.models import LinearHeadBENDR
from eegatscale.transforms import Standardize
from scipy.stats import ttest_ind
import torch
import tqdm

from captum.concept import TCAV, Concept
from captum.concept._utils.common import concepts_to_str
from concept_classifier import TCAVClassifier
import warnings
warnings.filterwarnings("ignore")

from dataset_utils import Experiment

In [20]:
concepts_dir = '/scratch/s194101/concepts/'

print(list(os.listdir(concepts_dir)))



['left_right_concepts_MMIDB_Gamma_new_T0', 'random_tuh_alpha_rh', 'random_mmidb_alpha_no_baseline_lh', 'random_mmidb_gamma_baseline_rh', 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma', 'random_tuh_alpha', 'random_mmidb_alpha_no_baseline_with_sigma', 'sanity_check_concepts_MMIDB_alpha_no_baseline', 'random_right_concept_MMIDB_Alpha_sanity_check_1000_no_baseline', 'random_mmidb_alpha_baseline_T0_rh', 'random_mmidb_gamma_no_baseline_rh', 'random_mmidb_alpha_lh', 'sanity_check_concepts_MMIDB_alpha_baseline_with_sigma', 'random_right_concept_MMIDB_Alpha_sanity_check', 'sanity_check_concepts_MMIDB_alpha_baseline', 'random_mmidb_gamma_baseline_T2', 'TUH_clean_alpha_subtract_Truesigma_Trueabs_100.0_142440_131124', 'random_mmidb_alpha_rh', 'random_right_concept_MMIDB_Alpha_sanity_check_1000', 'random_tuh_gamma_1000', 'TUSZ', 'random_mmidb_gamma_no_baseline_lh', 'random_mmidb_alpha_baseline_T0_lh', 'random_tuh_alpha_1000', 'random_mmidb_alpha_no_baseline', 'random_tuh_alpha_lh', 

## load model

In [21]:
out_features = 2
# encoder_path = "/scratch/agjma/checkpoints/tuh-noica-standardize-epoch=9-step=68317-val_loss=0.26.ckpt"
encoder_path = "/scratch/s194101/finetune_logs_all/mmidb_noica_noica_5.0_splits10_repeat2/version_19/checkpoints/epoch=7-step=8120.ckpt"

model = LinearHeadBENDR(encoder_path, encoder_h=512, in_features=19, out_features=out_features)
state_dict = torch.load(encoder_path, map_location=torch.device('cpu'))["state_dict"]
model.load_state_dict(state_dict)
model.eval()
print("model loaded")

Initialized mask embedding and position encoder from  /scratch/s194101/finetune_logs_all/mmidb_noica_noica_5.0_splits10_repeat2/version_19/checkpoints/epoch=7-step=8120.ckpt
model loaded


In [22]:
# target_data = "random_left_concept_MMIDB_Alpha_sanity_check"
# target_data = "random_mmidb_gamma_baseline_T1"
# target_data = "random_left_concept_MMIDB_Alpha_sanity_check_1000_no_baseline"
target_data = "random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma"
# target_data = "random_left_concept_MMIDB_Alpha"
# random_concept = "random_mmidb_alpha_new"
# random_concept = "random_mmidb_alpha_no_baseline_with_sigma"
# random_concept = "random_mmidb_alpha_no_baseline"
# random_concept = "random_mmidb_alpha_baseline_T0"
# random_concept = "random_mmidb_gamma_no_baseline"
# random_concept = "random_tuh_all_1000"
random_concept = "random_tuh_alpha_1000"
# random_concept = "random_tuh_gamma_1000"

concept_dir_names = [
    # "random_left_concept_MMIDB_Alpha_sanity_check",
    # "random_right_concept_MMIDB_Alpha_sanity_check",
    "random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma",
    "random_right_concept_MMIDB_Alpha_sanity_check_1000_with_sigma",
# #     # "random_left_concept_MMIDB_Alpha",
# #     # "random_right_concept_MMIDB_Alpha",
    "random_mmidb_alpha_lh",
# #     # "left_right_concepts_MMIDB_alpha_new/Alpha_lh",
# #     # "left_right_concepts_MMIDB_alpha_new_T0/Alpha_lh",
# #     # "random_tuh_alpha_lh",
# #     # "random_mmidb_alpha_baseline_with_sigma_lh",
# #     # "random_mmidb_alpha_no_baseline_T0_lh",
# #     # "random_mmidb_gamma_no_baseline_lh",
    "random_mmidb_alpha_rh",
# #     # "left_right_concepts_MMIDB_alpha_new/Alpha_rh",
# #     # "left_right_concepts_MMIDB_alpha_new_T0/Alpha_rh",
# #     # "random_tuh_alpha_rh",
# #     # "random_mmidb_alpha_baseline_with_sigma_rh",
# #     # "random_mmidb_alpha_no_baseline_T0_rh",
# #     # "random_mmidb_gamma_no_baseline_rh",
    ]

# T0 concepts
# concept_dir_names = [
#     "random_left_concept_MMIDB_Alpha_sanity_check",
#     "random_right_concept_MMIDB_Alpha_sanity_check",
#     "random_mmidb_alpha_baseline_T0_lh",
#     "random_mmidb_alpha_baseline_T0_rh",
#     # "random_tuh_alpha_lh",
#     # "random_tuh_alpha_rh",
# ]

# gamma
# concept_dir_names = [
#     "random_mmidb_gamma_baseline_T1",
#     "random_mmidb_gamma_baseline_T2",
#     "random_mmidb_gamma_baseline_lh",
#     "random_mmidb_gamma_baseline_rh",
# ]
# concept_dir_names = [
#     "sanity_check_concepts_MMIDB_alpha_no_baseline_with_sigma/Alpha_Premotor Cortex-lh",
#     "sanity_check_concepts_MMIDB_alpha_no_baseline_with_sigma/Alpha_Premotor Cortex-rh",
#     "sanity_check_concepts_MMIDB_alpha_no_baseline_with_sigma/Alpha_Somatosensory and Motor Cortex-lh",
#     "sanity_check_concepts_MMIDB_alpha_no_baseline_with_sigma/Alpha_Somatosensory and Motor Cortex-rh",
#     ]
# concept_dir_names = [
#     "sanity_check_concepts_MMIDB_alpha_no_baseline/Alpha_Premotor Cortex-lh",
#     "sanity_check_concepts_MMIDB_alpha_no_baseline/Alpha_Premotor Cortex-rh",
#     "sanity_check_concepts_MMIDB_alpha_no_baseline/Alpha_Somatosensory and Motor Cortex-lh",
#     "sanity_check_concepts_MMIDB_alpha_no_baseline/Alpha_Somatosensory and Motor Cortex-rh",
#     ]
experiments = []
idx = 0
for concept_dir_name in concept_dir_names:
    exp = Experiment(model,
                    target_data,
                    concepts_dir,
                    [
                    concept_dir_name, # this is where you add extra concepts, (in the square brackets)
                    random_concept,
                    ],
                    verbose=True,
                    n_concept_samples=50,
                    n_runs=100,
                    internal_concept_idx=idx)
    experiments.append(exp)
    idx += 1000

Loaded dataset random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma with 1000 samples
Loaded dataset random_tuh_alpha_1000 with 1000 samples
Loaded input dataset random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma with 1000 samples
Loaded dataset random_right_concept_MMIDB_Alpha_sanity_check_1000_with_sigma with 1000 samples
Loaded dataset random_tuh_alpha_1000 with 1000 samples
Loaded input dataset random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma with 1000 samples
Loaded dataset random_mmidb_alpha_lh with 200 samples
Loaded dataset random_tuh_alpha_1000 with 1000 samples
Loaded input dataset random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma with 1000 samples
Loaded dataset random_mmidb_alpha_rh with 200 samples
Loaded dataset random_tuh_alpha_1000 with 1000 samples
Loaded input dataset random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma with 1000 samples


In [23]:
experiments[0].internal_concept_idx

200

In [24]:
for concept_name, concepts in experiments[0].concepts.items():
    print(concept_name)
    print(concepts)
    break

random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma
[Concept(0, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_000'), Concept(1, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_001'), Concept(2, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_002'), Concept(3, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_003'), Concept(4, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_004'), Concept(5, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_005'), Concept(6, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_006'), Concept(7, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_007'), Concept(8, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_008'), Concept(9, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_009'), Concept(10, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_010'), Concept(11, 'random_left_concept_MMIDB_Alpha_sanity_check

In [25]:
experiments[0].concept_names[0]

'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma'

In [26]:
# for concept_name, concepts in experiments[0].concepts.items():
#     print(concept_name)
#     print(concepts)
#     for concept in concepts:
#         print(next(iter(concept.data_iter)).shape)
#         print(next(iter(concept.data_iter))[0,0])

# experiments[0].concepts.values()

In [27]:
layers = ["encoder", "enc_augment", "summarizer", "extended_classifier", "clf"]

def run_tcav(experiment:Experiment, layers:list):
    if "/" in experiment.concept_names[0]:
        save_path = f"/scratch/s194101/temp/cav/{experiment.concept_names[0].split('/')[-1]}/"
    else:
        save_path = f"/scratch/s194101/temp/cav/{experiment.concept_names[0]}/"
    tcav = TCAV(model, layers, model_id="BendrModel", classifier=TCAVClassifier(), save_path=save_path)
    # tcav = TCAV(model, layers, save_path=save_path)
    tcav_scores = tcav.interpret(inputs=experiment.inputs, experimental_sets=experiment.experimental_sets, target=0)
    tcav_scores = dict(tcav_scores)

    save_path = f"/home/s194101/Bachelors-explainability/{experiment.inputs_name}_{experiment.concept_names[0].split('/')[-1]}.pkl" if "/" in experiment.concept_names[0] else f"/home/s194101/Bachelors-explainability/{experiment.inputs_name}_{experiment.concept_names[0]}.pkl"
    with open(save_path, "wb") as f:
        pickle.dump(
            {"tcav_scores": tcav_scores,
             "experimental_sets": experiment.experimental_sets,},f)
    return tcav_scores
        

In [28]:
tcav = TCAV(model, layers, model_id="BendrModel", classifier=TCAVClassifier(), save_path="/scratch/s194101/temp/cav_test/")
tmp_scores = tcav.interpret(inputs=experiments[0].inputs, experimental_sets=[experiments[0].experimental_sets[0], experiments[1].experimental_sets[0]], target=0)


In [29]:
experiments[1].experimental_sets[0]

[Concept(1000, 'random_right_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_000'),
 Concept(1100, 'random_tuh_alpha_1000_000')]

In [30]:
experiments[0].experimental_sets[0]

[Concept(0, 'random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma_000'),
 Concept(100, 'random_tuh_alpha_1000_000')]

In [31]:
for exp in experiments:
    run_tcav(exp, layers)
    print(exp.inputs_name, exp.concept_names[0])

random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma
random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma random_right_concept_MMIDB_Alpha_sanity_check_1000_with_sigma
random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma random_mmidb_alpha_lh
random_left_concept_MMIDB_Alpha_sanity_check_1000_with_sigma random_mmidb_alpha_rh


In [32]:
all_experimental_sets = []
all_tcav_scores = []

for i, exp in enumerate(experiments):
    # where the files are saved
    file_path = f"/home/s194101/Bachelors-explainability/{exp.inputs_name}_{exp.concept_names[0].split('/')[-1]}.pkl" if "/" in exp.concept_names[0] else f"/home/s194101/Bachelors-explainability/{exp.inputs_name}_{exp.concept_names[0]}.pkl"
    with open(file_path, "rb") as f:
        data = pickle.load(f)
        all_tcav_scores.append(data["tcav_scores"])
        all_experimental_sets.append(data["experimental_sets"])

In [33]:
from plot_utils import assemble_scores, plot_tcav_scores
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, mannwhitneyu



def plot_tcav_scores_plotly(all_experimental_sets, all_tcav_scores, concept_names=None, title="", score_type="sign_count", alpha=0.05, only_significant=True,
                     with_error=True, plt_name="plot", file_type="svg"):
    tcav_score0 = next(iter(all_tcav_scores[0].values()))
    layers = list(tcav_score0.keys())
    scores_mean = {layer: [] for layer in layers}
    scores_std = {layer: [] for layer in layers}
    stars = {layer: [] for layer in layers}
    concept_short_names = []
    for experimental_sets, tcav_scores in zip(all_experimental_sets, all_tcav_scores):
        concept_short_codes = [int(str(concept.id)[:5]) for concept in experimental_sets[0]]
        concept_short_names_next = [concept.name[:-4] for concept in experimental_sets[0]]
        concept_short_names.append(concept_short_names_next[0])

        for layer in layers:
            pos_scores = assemble_scores(tcav_scores, experimental_sets, 0, layer, score_type)
            neg_scores = assemble_scores(tcav_scores, experimental_sets, 1, layer, score_type)
            # t-test
            _, pval = ttest_ind(pos_scores, neg_scores)
            # mann whitney u rank test
            _, pval_whit = mannwhitneyu(pos_scores, neg_scores)

            print(f"pval: {pval}, pval_whit: {pval_whit}")
            pval = pval_whit
            # print(pval)
            # Bonferroni correction
            m = 2
            alpha = alpha / m
            # Get non-significant and significant scores
            # not_significant = np.mean(pos_scores) < np.mean(neg_scores) or pval >= alpha
            not_significant = pval >= alpha
            stars[layer].append(~not_significant)
            if not_significant and only_significant:
                scores_mean[layer].append(0.0)
                scores_std[layer].append(0.0)
            else:
                scores_mean[layer].append(np.mean(pos_scores))
                scores_std[layer].append(np.std(pos_scores))

    if not concept_names:
        scores_mean_df = pd.DataFrame(data=scores_mean, index=concept_short_names)
        scores_std_df = pd.DataFrame(data=scores_std, index=concept_short_names)
    else:
        scores_mean_df = pd.DataFrame(data=scores_mean, index=concept_names)
        scores_std_df = pd.DataFrame(data=scores_std, index=concept_names)

    stars = {layer: ["*" if star else "" for star in stars[layer]] for layer in layers}

    fig = go.Figure()
    for layer in layers:
        fig.add_trace(go.Bar(
            x=concept_short_names,
            y=scores_mean[layer],
            # error_y=dict(type='data', array=scores_std[layer]),
            name=layer,
            text=stars[layer],
            # textposition='outside',
            # text color
        ))

    fig.update_layout(
        width=1200,
        height=720,
        # barmode='group',
        title=title,
        yaxis_title='TCAV score',
        xaxis_title='Concept',
        font=dict(
            size=18,
        ),

    )
    # print(stars)
    # change x tick labels
    fig.update_xaxes(tickvals=[i for i in range(len(concept_short_names))])
    fig.update_xaxes(ticktext=concept_names)

    # set y-axis to always go from 0 to 1
    fig.update_yaxes(range=[0, 1])
    # change tick angle
    # fig.update_xaxes(tickangle=0)
    # save figure
    fig.write_image(f"figures/{plt_name}_plotly.{file_type}")
    fig.show()




In [34]:
concept_names = ["Left fist movement", "Right fist movement", "Alpha left hemisphere", "Alpha right hemisphere"]
# concept_names = ["Left fist movement", "Right fist movement", "Gamma left hemisphere", "Gamma right hemisphere"]
# concept_names = ["Premotor Cortex-lh", "Premotor Cortex-rh", "Somatosensory and Motor Cortex-lh", "Somatosensory and Motor Cortex-rh"]
# concept_names = ["Early Visual Cortex-lh", "Early Visual Cortex-rh", "Primary Visual Cortex-lh", "Primary Visual Cortex-rh"]
title = "Left Fist Classifcation"
# plt_name = "left_fist_prediction_alpha_vs_random_tuh_alpha"
plt_name = "left_fist_prediction_alpha_baseline_with_sigma_vs_random_tuh_alpha"
# plt_name = "left_fist_prediction_anatomical_alpha_premotor_somatosensory_vs_random_mmidb_alpha_no_baseline"
# plt_name = "left_fist_prediction_anatomical_alpha_visual"

# scores_mean_df, scores_std_df = plot_tcav_scores(all_experimental_sets, all_tcav_scores, concept_names=concept_names, title=title, score_type="sign_count", alpha=0.05, only_significant=False, with_error=False, plt_name=plt_name, file_type="png")
# scores_mean_df
# scores_std_df   

plot_tcav_scores_plotly(all_experimental_sets, all_tcav_scores, concept_names=concept_names, title=title, score_type="sign_count", alpha=0.05, only_significant=False, with_error=False, plt_name=plt_name, file_type="png")

pval: 0.07796515567407286, pval_whit: 0.12011872864262427
pval: 0.004086917161948512, pval_whit: 0.005894504181361975
pval: 6.148943803721968e-26, pval_whit: 1.889850831898669e-21
pval: 1.5599780463119083e-16, pval_whit: 2.6119866607507058e-14
pval: 1.2526258651544202e-22, pval_whit: 2.2347902860245473e-18
pval: 8.369652651044192e-104, pval_whit: 9.662243448279715e-37
pval: 1.552586615147839e-129, pval_whit: 1.151884971453126e-39
pval: 1.8677426772070243e-121, pval_whit: 8.8332699252075e-39
pval: 6.144969430430456e-66, pval_whit: 2.2351404568528277e-35
pval: 8.581653085474213e-141, pval_whit: 1.8469846026480235e-43
pval: 1.3871398779915986e-16, pval_whit: 2.443864276577385e-15
pval: 1.6640036028953774e-15, pval_whit: 2.1512931757835593e-16
pval: 0.5755937418941943, pval_whit: 0.4801238474269688
pval: 0.15886959556739644, pval_whit: 0.15875743750830362
pval: 4.7738117806692215e-14, pval_whit: 1.76418326209705e-12
pval: 7.09651651281784e-57, pval_whit: 2.7269720270950447e-31
pval: 1.6906

In [35]:
# all_tcav_scores[2]
assemble_scores(all_tcav_scores[1], all_experimental_sets[1], 0, "encoder", "sign_count")

array([0.  , 0.03, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.11,
       0.  , 0.  , 0.  , 0.  , 0.48, 0.  , 0.06, 0.  , 0.  , 0.06, 0.  ,
       0.  , 0.03, 0.02, 0.  , 0.  , 0.  , 0.  , 0.02, 0.29, 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.01, 0.  , 0.  , 0.  , 0.01, 0.03, 0.97, 0.2 , 0.  , 0.  ,
       0.  , 0.32, 0.04, 0.09, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.03, 0.1 , 0.43, 0.02, 0.  , 0.  , 0.07, 0.  , 0.  , 0.  , 0.  ,
       0.01, 0.  , 0.  , 0.  , 0.03, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01,
       0.  , 0.  , 0.84, 0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.  ,
       0.  ], dtype=float32)